In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, datasets

In [3]:
import numpy as np

In [4]:
#from PIL import Image
from skimage import io, transform
import os

# Convert matlab data to jpg image

In [5]:
data_folder = "/home/user170/shared-data/Personal_Dev/Machine-Learning/Data/Depth/NYU-Depth-Dataset-V2/"

# Read Images

In [ ]:
# SCIKIT IMAGE USE
rgb_img = io.imread(os.path.join(data_folder, "Images/RGB/0000.jpg"))
d_img = io.imread(os.path.join(data_folder, "Images/Depth-TIFF/0000.tiff"))

In [ ]:
plt.imshow(rgb_img)

In [ ]:
plt.imshow(d_img)
print(np.max(d_img))
print(np.min(d_img))

In [6]:
class LoadData(Dataset):
    def __init__(self, root_dir, testset_ratio=0.3, transform=None):
        """
        Args:
            root_dir (string): Directory with RGB and Depth-TIFF images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        #self.mat_data = h5py.File(root_dir+mat_file, 'r')
        self.root_dir = root_dir
        self.transform = transform
        self.testset_ratio = testset_ratio
        
        self.trainset_idx, self.testset_idx, self.trainset_num, self.testset_num = self._seperate_dataset()

    def _seperate_dataset(self):
        """
        Seperate Train and Test dataset
        
        Returns: random indice of train test datasets, and length of those.
        """
        dataset_len = len(os.listdir(self.root_dir+"Images/RGB/"))
        rnd_idx = [i for i in range(dataset_len)]
        np.random.shuffle(rnd_idx)
        split_thresh = int(dataset_len*(1-self.testset_ratio))

        train_idx = rnd_idx[:split_thresh]
        test_idx = rnd_idx[split_thresh:]

        trainset_len = len(train_idx)
        testset_len = len(test_idx)

        return train_idx, test_idx, trainset_len, testset_len
    
    def __len__(self):
        self.dataset_len = len(os.listdir(self.root_dir+"Images/RGB/"))
        return self.dataset_len
    
    def __getitem__(self, idx):
        #rgb_img = Image.open(self.root_dir+'Images/RGB/{0:04d}.jpg'.format(self.trainset_idx[idx]))
        #depth_img = Image.open(self.root_dir+'Images/Depth-TIFF/{0:04d}.tiff'.format(self.trainset_idx[idx]))
        rgb_img = io.imread(self.root_dir+'Images/RGB/{0:04d}.jpg'.format(self.trainset_idx[idx]))
        depth_img = io.imread(self.root_dir+'Images/Depth-TIFF/{0:04d}.tiff'.format(self.trainset_idx[idx]))
            
        if self.transform:
            rgb_img = self.transform(rgb_img)
            depth_img = self.transform(depth_img)
        
        #print(np.max(rgb_img.numpy()), np.min(rgb_img.numpy()))
        #print(np.max(depth_img.numpy()), np.min(depth_img.numpy()))
        sample = {'RGB':rgb_img, 'DEPTH':depth_img}
        
        return sample

In [ ]:
dataloader = LoadData(data_folder)

In [ ]:
for i in range(len(dataloader)):
    sample = dataloader[i]
    print(i, 
          '[RGB u: {:.2f}, std: {:.2f}]'.format(np.mean(sample['RGB']), np.std(sample['RGB'])),
          '\t[DEPTH u: {:.2f}, std: {:.2f}]'.format(np.mean(sample['DEPTH']), np.std(sample['DEPTH'])))
    
    if i == 3:
        #plt.show()
        break

# Transformation

In [7]:
class ToTensor(object):
    """Convert ndarray in sample to Tensors."""
    
    def __call__(self, sample_img):
        if len(np.shape(sample_img)) > 2:
            # swap colour axis because
            # numpy image: H x W x C
            # torch image: C x H x W
            return torch.from_numpy(sample_img.transpose((2, 0, 1)))
            #rgb_img = np.moveaxis(np.asarray(rgb_img), [-1], [0])
        else:
            return torch.from_numpy(sample_img)

### Using class function

Transformation can be performed when class instantiation.

In [ ]:
load_data[0]['DEPTH'].shape

In [ ]:
composed = transforms.Compose([transforms.ToTensor()])
load_data = LoadData(root_dir=data_folder, transform=composed)

ax = plt.subplot(2, 1, 1)
plt.imshow(load_data[0]['RGB'].numpy().transpose((1, 2, 0)))
ax = plt.subplot(2, 1, 2)
plt.imshow(load_data[0]['DEPTH'][0].numpy())
print(np.max(load_data[0]['DEPTH'][0].numpy()))

# DataLoader

In [10]:
load_data = LoadData(root_dir=data_folder)

In [8]:
class Normalization(object):
    """Normalization for a given 2d array"""     
    def __call__(self, sample_img):        
        if len(np.shape(sample_img)) > 2:
            # RGB IMAGE NORMALIZATION W.R.T DIMENSIONS
            rgb_img_norm_ch0 = self._normalizer(sample_img[:, :, 0])
            rgb_img_norm_ch1 = self._normalizer(sample_img[:, :, 1])
            rgb_img_norm_ch2 = self._normalizer(sample_img[:, :, 2])
        
            rgb_img = np.stack((rgb_img_norm_ch0, rgb_img_norm_ch1, rgb_img_norm_ch2))
        
            # swap colour axis because
            # numpy image: H x W x C
            # torch image: C x H x W
            return rgb_img.transpose((1, 2, 0))
        
        else:
            # DEPTH IMAGE NORMALIZATION
            return self._normalizer(sample_img)
        
    def _normalizer(self, array_2d):
        array_scaled = (array_2d - np.min(array_2d)) / (np.max(array_2d) - np.min(array_2d))
        #print(np.max(array_scaled), np.min(array_scaled))
        
        return array_scaled

In [ ]:
np.shape(load_data[0]['DEPTH'])

In [11]:
norm = Normalization()
norm_img = norm(load_data[0]['DEPTH'])
print('MAX: {0}, MIN: {1}'.format(np.max(norm_img), np.min(norm_img)))

MAX: 1.0, MIN: 0.0


In [12]:
composed = transforms.Compose([Normalization(), ToTensor()])
load_data = LoadData(root_dir=data_folder, transform=composed)

In [16]:
load_data[0]

{'DEPTH': tensor([[0.7612, 0.7612, 0.7613,  ..., 0.1595, 0.1595, 0.1595],
         [0.7612, 0.7612, 0.7613,  ..., 0.1595, 0.1595, 0.1595],
         [0.7612, 0.7612, 0.7612,  ..., 0.1595, 0.1595, 0.1595],
         ...,
         [0.2507, 0.2507, 0.2508,  ..., 0.0534, 0.0534, 0.0535],
         [0.2507, 0.2508, 0.2508,  ..., 0.0533, 0.0534, 0.0534],
         [0.2508, 0.2508, 0.2508,  ..., 0.0533, 0.0534, 0.0534]]),
 'RGB': tensor([[[0.9569, 0.9882, 0.9569,  ..., 0.9529, 1.0000, 1.0000],
          [0.9647, 0.9882, 0.9608,  ..., 1.0000, 1.0000, 0.9608],
          [0.9882, 0.9882, 0.9843,  ..., 0.9843, 1.0000, 1.0000],
          ...,
          [0.9922, 0.9765, 1.0000,  ..., 0.9765, 0.9961, 0.9765],
          [1.0000, 0.9922, 0.9961,  ..., 0.9765, 0.9961, 0.9882],
          [1.0000, 0.9843, 0.9961,  ..., 0.9647, 0.9882, 1.0000]],
 
         [[0.9843, 1.0000, 0.9765,  ..., 0.9294, 1.0000, 1.0000],
          [0.9843, 1.0000, 0.9804,  ..., 0.9961, 1.0000, 0.9608],
          [1.0000, 1.0000, 0.992